# Deutch-Jozsa Algorithm
DJ is a quantum algorithm that exhibits an exponential speedup over classical implementation. Given n-qubits for n-number of parameters, its time complexity is O(1) compared to the worst case on classical implementation which is O(2^(n-1) + 1). Though it has no practical application, it clearly illustrates the quantum mechanical effects of superposition, interference and entanglement. 

In [ ]:
// NOTE: Namespace not accepted due to the non-contiguous nature of cells
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert as Convert;
open Microsoft.Quantum.Measurement; 
open Microsoft.Quantum.Arrays;

## 1-Qubit Oracles
Oracles are the functions which property we want to determine. Note the use of [ancilla](https://en.wikipedia.org/wiki/Ancilla_bit) qubit to respresent the result. This is a requirement in [reversible computing](https://en.wikipedia.org/wiki/Reversible_computing) like QC.

In [ ]:
operation Const0(qubits: Qubit[]) : Unit {
    // This is universal and can be used by any
    // n-qubit oracle. It's equal to doing nothing.
    ApplyToEach(I, qubits); 
}

operation Const1(qubits: Qubit[]) : Unit {       
    let indexOfAncilla = Length(qubits)-1;
    X(qubits[indexOfAncilla]);      
}

operation Balanced0(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
}

operation Balanced1(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[1]);
    X(qubits[1]);
}

## 2-Qubit Oracles

In [ ]:
operation Balanced_2Qubit0(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[2]);
}

operation Balanced_2Qubit1(qubits: Qubit[]) : Unit {
    CNOT(qubits[0], qubits[2]);
    X(qubits[2]);
}

operation Balanced_2Qubit2(qubits: Qubit[]) : Unit {
    CNOT(qubits[1], qubits[2]);        
}

operation Balanced_2Qubit3(qubits: Qubit[]) : Unit {
    CNOT(qubits[1], qubits[2]);  
    X(qubits[2]);
}

operation Balanced_2Qubit4(qubits: Qubit[]) : Unit {
    CNOT(qubits[0],qubits[2]);
    CNOT(qubits[1],qubits[2]);
}

operation Balanced_2Qubit5(qubits: Qubit[]) : Unit {
    CNOT(qubits[0],qubits[2]);
    CNOT(qubits[1],qubits[2]);
    X(qubits[2]);
}

## Circuit
To accomodate different oracles, we isolate the circuit creation in its own operation. Note that Q# has no support on named delegate, only lambda.

In [ ]:
operation DeutchJozsa(bits: Int, oracle: ((Qubit[]) => Unit)) : Result {
    use qubits = Qubit[bits + 1] {

        X(qubits[bits]);

        ApplyToEach(H,qubits);

        oracle(qubits);

        ApplyToEach(H,qubits);            

        mutable result = Zero;   
        // Hardcoded logic to cater only for 2-qubit oracle      
        if  bits == 2 {       
            ApplyToEach(X, [qubits[0], qubits[1]]);
            CCNOT(qubits[0], qubits[1], qubits[2]);               
        }

        set result = M(qubits[bits-1]);

        ResetAll(qubits);

        return result;
    }   
}

## Trying out Types

In [ ]:
// NOT WORKING! Bug?
//  https://docs.microsoft.com/en-us/azure/quantum/user-guide/language/expressions/itemaccessexpressions#item-access-for-user-defined-types

newtype DJParams = (
                        QBitCount : Int,
                        Oracles   :((Qubit[]) => Unit)[] // Array of void delegates accepting Qubit array as param                                                         
                    );

newtype Complex = (Real: Double, Imaginary : Double);
//let complex = Complex(1.,0.);

In [ ]:
@EntryPoint()
operation DJMain() : Result[] {

        // 1-qubit oracles        
        let oracles1Qubit = [Const0, Const1, Balanced0, Balanced1];
        
        // 2-qubit oracles      
        // CAUTION! This causes error when submitting all items. 
        // Still trying to find the max acceptable number. BUG????
        let oracles2Qubit = [Const0, Const1];
                        //, Balanced_2Qubit0
                        //, Balanced_2Qubit1
                        //, Balanced_2Qubit2
                        //, Balanced_2Qubit3
                        //, Balanced_2Qubit4
                        //, Balanced_2Qubit5];

        // 1-qubit params
        let oracles = oracles1Qubit;
        let qbitCount = 1;

        // 2-Qubit params
        //let oracles = oracles2Qubit;
        //let qbitCount = 2;

        mutable results = ConstantArray(Length(oracles),Zero);
        
        for i in 0 .. Length(oracles)-1 {            
            set results w/= i <- DeutchJozsa(qbitCount,oracles[i]);          
        }               

        return results;
}

In [ ]:
%simulate DJMain

In [ ]:
%estimate DJMain

In [ ]:
%azure.connect "<resource id here which you can find in the Overview tab of your Quantum Workspace>"

In [ ]:
%azure.target ionq.qpu

In [ ]:
%azure.submit DJMain jobName="DJ 3 Qubits 8 items" shots="1024"

In [ ]:
%azure.status "job id GUID you just submitted"

In [ ]:
%azure.output "job id GUID you just submitted. Do this if the status is not WAITING anymore"